In [1]:
#importing dependencies
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [7]:
#reading csv file
df = pd.read_csv('/home/sumit/PriceLens/data/processed/cardekho_imputated.csv')

In [8]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
df.head()

,brand,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [10]:
#defining independent and dependent features
X = df.drop(['brand', 'selling_price'], axis=1)
y = df['selling_price']

### Feature Engineering and Scaling

One Hot Encoding for categorical columns with lesser Unique values and not ordinal

In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
ohe = OneHotEncoder()
le = LabelEncoder()
scaler = StandardScaler()

In [14]:
ohe_cols = ['seller_type', 'fuel_type', 'transmission_type']
scaler_cols = X.select_dtypes(exclude='object').columns     #numerical columns

In [16]:
#creating Column Transformer with 3 types of transformers
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", ohe, ohe_cols),
        ("StandardScaler", scaler, scaler_cols)
    ], remainder="passthrough"
)

In [17]:
X.head()

,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [18]:
X = preprocessor.fit_transform(X)

In [19]:
#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
X.shape

(14556, 16)

### Model Training and Selection

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [22]:
##Creating Function to Evaluate Model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [23]:
## Selecting best model
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
   
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 239328.7269
- Mean Absolute Error: 156187.0310
- R2 Score: 0.7320
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 240261.5554
- Mean Absolute Error: 157140.4201
- R2 Score: 0.7221


Lasso
Model performance for Training set
- Root Mean Squared Error: 239328.7393
- Mean Absolute Error: 156186.8957
- R2 Score: 0.7320
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 240259.5561
- Mean Absolute Error: 157137.0434
- R2 Score: 0.7221


Ridge
Model performance for Training set
- Root Mean Squared Error: 239334.3916
- Mean Absolute Error: 156202.2980
- R2 Score: 0.7320
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 240234.5470
- Mean Absolute Error: 157095.2220
- R2 Score: 0.7222


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 122578.1965
- Mean 

### Observation 

Random Forest Regressor and K-Neighbours Regressor have High accuracy score(r2_score) and comparitively less error. So these are the selected models

In [24]:
#crossvalidating the model
#Initialize few parameter for Hyperparamter tuning
knn_params = {"n_neighbors": [2, 3, 10, 20, 40, 50]}
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}

# Models list for Hyperparameter tuning
randomcv_list = [('KNN', KNeighborsRegressor(), knn_params),
                   ("RF", RandomForestRegressor(), rf_params)
                   
                   ]

In [ ]:
##Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_list:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
## Retraining the models with best parameters
models = {
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, min_samples_split=2, max_features=5, max_depth=None, 
                                                     n_jobs=-1),
     "K-Neighbors Regressor": KNeighborsRegressor(n_neighbors=10, n_jobs=-1)
    
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')